In [1]:
import syft as sy
import matplotlib.pyplot as plt
import torch
import torchvision
import time
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from time_tracker import time_tracker

c:\users\user\appdata\roaming\python\python36\site-packages\syft\lib\python\namedtuple.py:237: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:2897.)
  eig = x.eig(True)
c:\users\user\appdata\roaming\python\python36\site-packages\syft\lib\python\namedtuple.py:243: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To

AttributeError: type object 'Tensor' has no attribute 'fft'

# Setup

In [ ]:
monitor = time_tracker(interval = 5)

## Sets the name of the saved data, based on device and resource state, and the number of models to create

In [ ]:
test_name = 'Stationary_WIFI'
rpi_name = 'RPi8_1500'
num_of_models = 10

Based on Openmined's course: 
https://courses.openmined.org/courses/foundations-of-private-computation

The code for the Controller used in course can be found here: 
https://github.com/OpenMined/courses/blob/foundations-of-private-computation/federated-learning/duet_mnist/MNIST_Syft_Data_Scientist.ipynb

In [ ]:
#duet_RPi8_1500 = sy.join_duet(target_id="438be5a282e90f571b435feae2d0b648", network_url="http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000")
duet = sy.join_duet(target_id="2bd79a82ed05fb869cf1482c9f24455d")

In [ ]:
remote_torch = duet.torch

# Linear Regression

In [ ]:
#Linear Regression Model Dimensions (each image is 8x8, with 10 labeles, and 1437 samples)
in_dim = 64
out_dim = 10
n_samples = 1437

#LR Model traning Parameters
learning_rate = 0.01

iteration = 100

#Arrays required to compute RCoin Values

RCoins = []

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref = torch_ref)
        self.layer1 = self.torch_ref.nn.Linear(in_dim, 128)
        self.layer2 = self.torch_ref.nn.Linear(128, 256)
        self.layer3 = self.torch_ref.nn.Linear(256, 50)
        self.dropout1 = self.torch_ref.nn.Dropout(0.25)
        self.out = self.torch_ref.nn.Linear(50, out_dim)

    def forward(self, x):
        #with profiler_ref.record_function("Forward Pass"):
        x = self.torch_ref.nn.functional.relu(self.layer1(x))
        x = self.torch_ref.nn.functional.relu(self.layer2(x))
        x = self.torch_ref.nn.functional.relu(self.layer3(x))
        output = self.torch_ref.nn.functional.log_softmax(self.out(x), dim=1)
        return output

In [ ]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr, monitor): #profiler_ref

    losses = []

    for i in range(iterations):
        iter_start_time = time.time()
        optim.zero_grad()
        #with profiler_ref.profile(profiler_memory=True) as prof:
        output = model(data_ptr)

        # nll_loss = negative log-liklihood loss
        loss = torch_ref.nn.functional.nll_loss(output, target_ptr.long())

        loss_item = loss.item()
        
        loss_value = loss_item.get( reason="To evaluate training progress", request_block=True, timeout_secs=5 )
        
        if i % 10 == 0:
            if loss_value is not None:
                print("Train Epoch: {} loss {:.4}".format(i, loss_value))
            else:
                print("Train Epoch: {}".format(i))

        losses.append(loss_value)
        
        loss.backward()

        optim.step()
        
        x =  time.time() - iter_start_time
        monitor.QOS =  x
        monitor.QOS_list.append(x)

    return losses

In [ ]:
#Create the model and pass in our local copy of torch
local_model = SyNet(torch) #profiler
print(local_model)

## Receive Remote Data Pointer

In [ ]:
data_ptr = duet.store[0]
target_ptr = duet.store[1]

In [ ]:
mnsit = datasets.load_digits()
X, y = mnsit.data, mnsit.target

_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

X_test = torch.FloatTensor(np.array(X_test))
y_test = torch.FloatTensor(np.array(y_test))

## Send Models

In [ ]:
remote_optims, remote_models = [], []
for m in range(num_of_models):
    
    remote_models.append(local_model.send(duet))
    remote_optims.append(remote_torch.optim.SGD(params=remote_models[m].parameters(), lr=learning_rate))
    
    
print(remote_models)    
print(remote_optims)


## Run Training and track QOS values

In [ ]:
print("Training on {} - {}".format(rpi_name,test_name))
startTimes, trainingTimes, training_losses = [], [], []      
model_number = 1
monitor.run_monitor_thread()

for r in tqdm(range(num_of_models)):
    monitor.task = 'Linear_Regression_Training'
    monitor.model_num = model_number
    print("Round number:", r+1)
    startTimes.append(time.time())
    
    training_loss  = train(iteration, remote_models[r], remote_torch, remote_optims[r], data_ptr, target_ptr, monitor) #remote_profiler
    training_losses.append(training_loss)

    trainingTimes.append(time.time() - startTimes[r])
    print('Training time:', trainingTimes[-1],'for model', model_number)
    model_number += 1
    
print('Done - Stopping Monitoring thread')
time_data = monitor.stop_monitor_thread()
monitor.model_num = None
monitor.QOS = 0
monitor.QOS_list = [0]
monitor.task = 'None'

## Save results

In [ ]:
time_data.to_csv('data/{}/QOS_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))

# Rcoin 

## Receive Trained Models

In [ ]:
def get_local_model(model):

    local_model = model.get(
        request_block=True,
        reason="To run test and inference locally",
        timeout_secs=5,
    )

    return local_model

In [ ]:
local_models = []
for m in range(num_of_models):
    local_models.append(get_local_model(remote_models[m]))

## Accuracy of models on test data (section of data not used to train model on Worker deivce)

In [ ]:
count = 1
accuracy = []
for model in local_models:
    correct = 0
    preds = []
    
    print("Test Model", count)
    count += 1
    
    with torch.no_grad():
        
        for i in tqdm(range(len(X_test))):
        
            sample = X_test[i]
            y_hat = model(sample.unsqueeze(0))
            pred = y_hat.argmax().item()
            
            if y_test[i] == pred:
                correct += 1
                
            preds.append(pred)
            
    accuracy.append(accuracy_score(y_test, preds))

print(accuracy)

In [ ]:
def get_RCoinP(RCoins,num_of_models):

    RCoinPs = []
    RCoinPs.append(RCoins[0])
    for i in range(1,num_of_models):
        if i <= 4:
            RCoinPs.append(RCoinPs[i-1] + RCoins[i])
        else:
            RCoinPs.append(RCoinPs[i-1] + RCoins[i] -  RCoins[i-5])
            
    return RCoinPs

In [ ]:
RCoins, training_loss_last = [], []
for i in range(num_of_models):
    training_loss_last.append(training_losses[i][-1])
    RCoins.append( (iteration * accuracy[i]) / (training_losses[i][-1] * trainingTimes[i] * learning_rate))

In [ ]:
print(RCoins)

In [ ]:
RCoinPs = get_RCoinP(RCoins,num_of_models)    

In [ ]:
out_dict = {'RCoin':RCoins,'RCoinP':RCoinPs, 'Accuracy':accuracy,'Training Losses': training_loss_last,'Training Times': trainingTimes}

out = pd.DataFrame(out_dict)
out.to_csv("data/{}/RCoin_{}_{}.csv".format(rpi_name,rpi_name, test_name), index=False)

# Matrix Multiplication

In [ ]:
def SyMatMull(torch_ref, monitor, n = 512, run_time = 300):
    
    #FLOPS_SCALE_FACTOR = 100_000_000_000
    # this represents the number of floating point computations per matrix mult
    ops = (n**3) + ((n-1)*(n**2)) # // n^2*(n-1) additions and n^3 mults
    
    monitor.task = 'Matrix_Multiplication'
    matrix1 = torch.randn(size=(n,n), dtype=torch.float32)
    matrix2 = torch.randn(size=(n,n), dtype=torch.float32)

    matrix1_pointer = matrix1.send(duet, pointable=True)
    matrix1_pointer = matrix2.send(duet, pointable=True)
    print('Encryted matrixes sent')
    
    matrixOut = torch_ref.matmul(matrix1_pointer, matrix1_pointer)

    #start = time.time()
    #num_iters = 0

    # runs the loop for the given amount of time
    while (time.time()- run_time < start):
        start_matMull = time.time()
        matrixOut = torch_ref.matmul(matrix1_pointer, matrix2_pointer)
        monitor.QOS = time.time() - start_matMull
        monitor.QOS_list.append(time.time() - start_matMull)
        
        #other QOS measure
        #num_iters += 1
        #monitor.QOS = (num_iters * ops / ( time.time() - start )) / FLOPS_SCALE_FACTOR
        #monitor.QOS_list.append(( num_iters * ops / ( time.time() - start ) ) / FLOPS_SCALE_FACTOR )

    del matrix1, matrix2

    

    # calculates the rate at which the CPU performs floating point computations
    #cpu_rate = num_iters * ops / run_time

    #return cpu_rate/FLOPS_SCALE_FACTOR

In [ ]:
monitor.run_monitor_thread()
SyMatMull(remote_torch, monitor)
monitor.model_num = None
monitor.QOS = 0
monitor.QOS_list = [0]
monitor.task = 'None'
#time_data = monitor.stop_monitor_thread()
#time_data.to_csv('data/microbench/{}/QOS_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))

# Inverse Function

def SyCholeskyInverse(torch_ref, monitor, n = 512, run_time = 300):
    
    
    monitor.task = 'Cholesky_Inverse'
    matrix = torch.randn(size=(n,n), dtype=torch.float32)
    matrix = torch.mm(matrix, matrix.t()) + 1e-05 * torch.eye(3)
    matrixIn = torch.cholesky(matrix)

    matrixIn_pointer = matrixIn.send(duet, pointable=True)
    print('Encryted matrixes sent')
    
    matrixOut = torch_ref.cholesky_inverse(matrixIn_pointer)

    # runs the loop for the given amount of time
    while (time.time()- run_time < start):
        start_cholInv = time.time()
        matrixOut = torch_ref.cholesky_inverse(matrixIn_pointer)
        monitor.QOS = time.time() - start_cholInv
        monitor.QOS_list.append(time.time() - start_cholInv)       

    del matrix

In [ ]:
def SyInverse(torch_ref, monitor, n = 512, run_time = 300):
    
    # attempt to calculate flops
    #FLOPS_SCALE_FACTOR = 100_000_000_000
    # this represents the number of floating point computations per matrix mult
    #matMulOps = (n**3) + ((n-1)*(n**2)) # // n^2*(n-1) additions and n^3 mults
    #LUFactorizationOps = 2 * matMulOps
    #lowerTriMatInv = n + n**2 + n**2 # n + n**2 multiplcationas and n**2 additions 
    #upperTriMatInv = 2*n + n**2
    #matInvOps =   LUFactorizationOps  + LUInversion + matMulOps
    # the inverse function is composed of an LU factorization (via a mat mul of three matrixes, L us a lower triangular matrix and U in an upper trainagular matrix) using     # the getrf routine and and the inverse of an LU-factored general matrix         
    # determined by getri routine. The routines belong to the mkl::lapack namespace
    # The getri function computes the inverse of both the triangle matrixes L and U by n^2 and then multiplies the inverses to come up witht the result

    monitor.task = 'Inverse'
    matrix = torch.randn(size=(n,n), dtype=torch.float32)
    
    matrix_pointer = matrix.send(duet, pointable=True)
    print('Encryted matrixes sent')
    
    matrixOut = torch_ref.inverse(matrix_pointer)

    #start = time.time()
    #num_iters = 0

    # runs the loop for the given amount of time
    while (time.time()- run_time < start):
        start_Inv = time.time()
        matrixOut = torch_ref.inverse(matrix_pointer)
        monitor.QOS = time.time() - start_Inv
        monitor.QOS_list.append(time.time() - start_Inv) 
                
        #num_iters += 1
        #monitor.QOS = (num_iters * ops / ( time.time() - start )) / FLOPS_SCALE_FACTOR
        #monitor.QOS_list.append(( num_iters * ops / ( time.time() - start ) ) / FLOPS_SCALE_FACTOR )

    del matrix

    # calculates the rate at which the CPU performs floating point computations
    #cpu_rate = num_iters * ops / run_time

    #return cpu_rate/FLOPS_SCALE_FACTOR

In [ ]:
monitor.run_monitor_thread()
cpu_rate = SyCholeskyInverse(remote_torch, monitor)
print(cpu_rate)
monitor.model_num = None
monitor.QOS = 0
monitor.QOS_list = [0]
monitor.task = 'None'
#time_data = monitor.stop_monitor_thread()
#time_data.to_csv('data/microbench/{}/QOS_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))

# Array Access

In [ ]:
def SyArrAccess(duet, monitor, arrLength = 100000, arrElementMax = 999999):
    
    monitor.task = 'Array Access'
    
    seed = int(time.time())
    np.random.seed(seed)
    
    arr = np.empty(shape=(arrLength,2))
    indexArr = np.arange(arrLength)
    np.random.shuffle(indexArr)
    
    for i in range(arrLength):
        arr[i,0] = np.random.randint(arrElementMax)
        arr[i,1] = indexArr[i]
    
    arr = torch.FloatTensor(arr)
    
    arr = arr.tag("Send Tensor")
    arr = arr.describe("This 2D tensor contains randomized elements and access indexes for memory test with length " + str(arrLength))

    arr_ptr = arr.send(duet, pointable=True)
    print('Tensor sent')
    
    searchFlag = True
    print('Waiting to receive result')
                   
    while searchFlag == True:
        for ptr in duet.store:
            if ptr.tags[0] == 'Return Tensor':
                ptr.request(reason = "Why I ought'a ...")
                time.sleep(5)
                squencedArrInfo = ptr.get()
                searchFlag = False
                print('Result received')  
                   
                return arr, arr_ptr, squencedArrInfo
            
def seqArrayCheck(arr, squencedArrInfo):
    
    squencedArrInfoNumpy = squencedArrInfo.numpy()
    arrNumpy = arr[:,0].numpy()
    indexArrNumpy = arr[:,1].numpy()

    index = 0
    squencedArr = squencedArrInfoNumpy[:,2]
    
    for i in indexArrNumpy:
        if arrNumpy[int(i)] != squencedArr[index]:
            print('Mismatch Error')
            return False
        index += 1
    print("Arrays Match")
    return True

In [ ]:
time_stamp_list = []

monitor.run_monitor_thread()
arr, arr_ptr, squencedArrInfo = SyArrAccess(duet, monitor, 100000, 999999)
if seqArrayCheck(arr, squencedArrInfo) == True:
    
    squencedArrInfoNumpy = squencedArrInfo.numpy()
    
    for timeStamp in squencedArrInfoNumpy[:,0]:
    
        timeObj = time.localtime(timeStamp)
        # creates time stamp Day-Month-Year Hour:Minute:Second
        time_stamp_list.append(str(timeObj.tm_mday) + '-' + str(timeObj.tm_mon) + '-' + str(timeObj.tm_year) + ' ' + str(timeObj.tm_hour) + ':' + str(timeObj.tm_min) + ':' + str(timeObj.tm_sec))
     
    squencedArrInfoDF = pd.DataFrame.from_dict({'time_stamp':time_stamp_list, 'time':squencedArrInfoNumpy[:,0],'memory_access_time':squencedArrInfoNumpy[:,1]})
    squencedArrInfoDF = squencedArrInfoDF.set_index('time_stamp')
    
    monitor.model_num = None
    monitor.QOS = 0
    monitor.QOS_list = [0]
    monitor.task = 'None'
    time_data = monitor.stop_monitor_thread()
    time_data.to_csv('data/microbench/{}/QOS_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))
    squencedArrInfoDF.to_csv('data/microbench/{}/ArrInfo_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))